######Data Curation/Processing

In [0]:
######Select/ Filter and derive flags/columns

In [0]:
#from pyspark.sql.functions import *
#select we have done prior 
#filter is same as where condition
curateddf = spark.read.csv("/Volumes/telecom_catalog_assign/landing_zone/landing_vol/BB_Practice data/custsmodified_DEnriched/part-00000-tid-3092141716748428475-337ed636-e169-49dd-9013-d3ce17d167cb-249-1-c000.csv",header=True,mode="permissive")
display(curateddf)
curateddf.printSchema()

#DSL operation
filterdf = curateddf.filter((col("age")>40) & (col("age")<60))
filterdf = curateddf.where((col("age")>40) & (col("age")<60))
display(filterdf.take(5))

#sql
filterdf = curateddf.filter("age>40 and age<60")
filterdf = curateddf.where("age>40 and age<60")
display(filterdf.take(5))

curateddf = curateddf.na.replace("2025-01-30","2025-12-30").withColumn("profession",initcap("profession"))

#derive flag
#DSL syntax: when(conditions,"value").when(conditions,"value2").otherwise("valuen").alis("colname")
curateddf1 = curateddf.select("*",when(col("age").isNull(),"U").
                              when((col("age")<=10),"C").
                              when((col("age")>10) & (col("age")<=19),"T").
                              when((col("age")>19) & (col("age")<=30),"A").
                              when((col("age")>30) & (col("age")<=50),"M").
                              otherwise("S").alias("agecatflag"))
display(curateddf1.take(10))

#SQL syntax: case when conditions
curateddf2 = curateddf1.selectExpr("*","""case when age is null then 'unknown'
                                  when age<=10 then 'child'
                                  when age>10 and age<=19 then 'Teenager'
                                  when age>19 and age<=30 then 'Adult'
                                  when age>30 and age<=50 then 'Middleage'
                                  else 'seniorcitizen' end as agecat""")
display(curateddf2.take(50))

#deriving columns
curateddf3 = curateddf2.select("*",datediff("datadt","loaddt"),
                               year("datadt").alias("datayear"),
                               month("datadt").alias("datamonth"),
                               last_day("datadt").alias("datalastday")
                               ).withColumn("agecat",initcap("agecat"))
display(curateddf3.take(10))



Groupby & Aggregate, orderby & limit

In [0]:
display(curateddf3.count())
#single groupby condition 
curateddf4 = curateddf3.groupby("profession").count()
display(curateddf4.take(20))

curateddf4 = curateddf3.withColumn("custid",col("custid").cast("long")).withColumn("age",col("age").cast("short"))
curateddf5.printSchema()

#multiple groupby/aggregate function
curateddf5=curateddf4.groupBy("profession").avg("age").withColumnRenamed("avg(age)","avgage")# single groupby & agg
display(curateddf6.take(100))

curateddf5 = curateddf4.groupby("profession").agg(count("custid").alias("custcount"),avg("age").alias("avgage"))#single groupby and multiple agg
display(curateddf5.take(100))

curateddf5 = curateddf4.groupby("profession","agecat").agg(count("custid").alias("custcount"),avg("age").alias("avgage"))#multiple groupby and agg
display(curateddf5.take(100))

curateddf6 = curateddf5.orderBy("profession","agecat")
display(curateddf6.take(20))

curateddf6.limit(20).show(10)

curateddf6.limit(50).filter("profession='Accountant'").show()
display(curateddf6.limit(50).filter("profession='Accountant'"))